In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

Load anndata

In [2]:
adata = sc.read_h5ad("test.h5ad")

In [3]:
# lognorm should be the default (expected)
adata.X = adata.layers['logcounts']

Parameters

In [4]:
groupby='label'
de_method='wilcoxon'
resource_name = 'consensus'
complex_policy = 'min'
key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']
verbose = False

In [5]:
# only relevant if perms are done
n_perms = 1000
seed = 69

Run LIANA PIPE

In [6]:
from liana import liana_pipe

In [7]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    de_method=de_method,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    resource=None)

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [8]:
lr_res.sort_values('ligand_means')

,ligand,ligand_complex,ligand_logfc,ligand_means,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfc,receptor_means,receptor_zscores,source,target,ligand_means_sums,receptor_means_sums
2288,CSF1,CSF1,-0.044315,0.000000,-0.105409,0.199056,CSF2RA,CSF2RA,-0.055999,0.000000,-0.105409,CD8 T,CD8 T,0.035162,0.040544
2096,BTLA,BTLA,-0.048226,0.000000,-0.105409,0.199056,TNFRSF14,TNFRSF14,-0.520295,0.294117,-0.250196,CD8 T,CD8 T,0.037057,1.424242
2087,IL18BP,IL18BP,-0.132456,0.000000,-0.145695,0.199056,IL6R,IL6R,-0.068199,0.000000,-0.105409,CD8 T,CD8 T,0.087242,0.045404
2086,IL6,IL6,-0.315346,0.000000,-0.149471,0.199056,IL6ST,IL6R_IL6ST,-0.121447,0.000000,-0.146079,CD8 T,CD8 T,0.140459,0.083360
2085,IL6,IL6,-0.315346,0.000000,-0.149471,0.199056,IL6R,IL6R_IL6ST,-0.068199,0.000000,-0.105409,CD8 T,CD8 T,0.140459,0.045404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,B2M,B2M,0.595277,5.810702,0.728883,0.199056,CD247,CD247,-2.154735,0.269538,-0.764050,NK,B,16.490938,3.277351
4178,B2M,B2M,0.595277,5.810702,0.728883,0.199056,CD3G,CD3G,-0.160309,0.196046,-0.106673,NK,NK,16.490938,0.797412
4332,B2M,B2M,0.595277,5.810702,0.728883,0.199056,KLRC1,KLRC1,1.285116,0.642471,0.494169,NK,NK,16.490938,0.893647
2647,B2M,B2M,0.595277,5.810702,0.728883,0.199056,CD3D,CD3D,2.097606,2.190063,0.951525,NK,CD8 T,16.490938,3.070996


Consensus

In [9]:
from liana.steady.scores.consensus import consensus

In [10]:
from functools import reduce
from scipy.stats import rankdata, beta

In [11]:
from liana.steady import cellphonedb, natmi, connectome, logfc, singlecellsignalr as sca

In [12]:
from liana.steady.aggregate import _robust_rank_aggregate, _rank_aggregate

In [13]:
_methods = [cellphonedb, connectome, logfc, natmi, sca]

In [14]:
adata = consensus(adata, groupby='label')

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


CellPhoneDB
Connectome
log2FC
NATMI
SingleCellSignalR


In [15]:
adata.uns['liana_res']

,source,target,ligand_complex,receptor_complex,lr_means,pvals,expr_prod,scaled_weight,lr_logfc,expr_prod_duplicated,spec_weight,lrscore,ranked_magnitude,ranked_specificity,ranked_steady
1589,B,CD8 T,HLA-DPB1,LAG3,2.009922,0.0,2.556716,0.918130,2.194075,2.556716,0.623592,0.889292,0.013062,0.000017,9.862040e-07
4009,NK,NK,SPON2,ITGB2,1.910813,0.0,3.563785,0.900085,2.289385,3.563785,0.514403,0.904614,0.023017,0.000017,9.862040e-07
1591,B,CD8 T,HLA-DRB5,LAG3,1.337915,0.0,1.492111,0.886746,2.004025,1.492111,0.723478,0.859876,0.083432,0.000017,9.862040e-07
1590,B,CD8 T,HLA-DPA1,LAG3,1.807187,0.0,2.235539,0.929952,2.229908,2.235539,0.649385,0.882509,0.032666,0.000017,9.862040e-07
1588,B,CD8 T,HLA-DQB1,LAG3,1.282576,0.0,1.404443,0.902600,2.235832,1.404443,0.808777,0.856189,0.088410,0.000017,9.862040e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,B,CD8 T,TNFSF13B,TNFRSF13C,1.000000,1.0,0.000000,-0.063277,-0.058579,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00
1759,B,CD8 T,TGFB1,TGFBR1_TGFBR2,1.000000,1.0,0.000000,-0.196001,-0.534732,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00
1760,B,CD8 T,TGFB1,ACVR1_TGFBR1_TGFBR2,1.000000,1.0,0.000000,-0.196001,-0.534732,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00
1730,B,CD8 T,NRG1,SIGLEC7,1.000000,1.0,0.000000,-0.144956,-0.173359,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00


In [17]:
adata.uns['liana_res'].sort_values('ranked_magnitude')

,source,target,ligand_complex,receptor_complex,lr_means,pvals,expr_prod,scaled_weight,lr_logfc,expr_prod_duplicated,spec_weight,lrscore,ranked_magnitude,ranked_specificity,ranked_steady
1584,B,CD8 T,HLA-DRA,LAG3,2.358596,0.000,3.109091,1.009779,2.698093,3.109091,0.737411,0.898561,0.004563,0.000001,9.862040e-07
1586,B,CD8 T,HLA-DQA1,LAG3,1.504316,0.000,1.755726,1.005434,2.494080,1.755726,0.894548,0.869394,0.065202,0.000005,9.862040e-07
1585,B,CD8 T,HLA-DQA2,LAG3,1.030748,0.000,1.005493,0.904647,2.088210,1.005493,0.966528,0.834368,0.128734,0.000014,2.747713e-06
316,B,B,BTLA,CD79A,1.485253,0.000,0.108705,0.752452,2.123316,0.108705,0.934813,0.623542,0.435257,0.000016,4.903092e-05
1592,B,CD8 T,HLA-DQB2,LAG3,0.459785,0.000,0.100964,0.543476,1.015481,0.100964,0.745453,0.614832,1.000000,0.000017,1.041830e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,CD8 T,NK,EFNA3,EPHA1,1.000000,1.000,0.000000,-0.127536,-0.116640,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00
3619,CD8 T,NK,EFNA4,EPHA4,1.000000,1.000,0.000000,-0.169616,-0.201252,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00
3139,B,NK,LTA,RIPK1,0.044245,0.925,0.001933,-0.132416,-0.287311,0.001933,0.023367,0.180919,1.000000,1.000000,1.000000e+00
3143,B,NK,EFNA3,EPHA1,1.000000,1.000,0.000000,-0.127536,-0.116640,0.000000,0.000000,0.000000,0.866153,1.000000,1.000000e+00


In [ ]:
if False is not False:
    print(1)

In [ ]:
adata.uns['liana_res']['CellPhoneDB']

In [ ]:
_key_cols = key_cols

In [ ]:
lr_res = adata.uns['liana_res'].copy()

In [ ]:
def aggregate(lr_res, _methods,
              _key_cols = ['source', 'target','ligand_complex', 'receptor_complex'],
              aggregate_method = 'rra',
              consensus_opts = ['Magnitude', 'Specificity', 'Steady']):
    # join the scores to the whole universe between the methods
    shared_keys = [
        lr_res[method.method_name].drop_duplicates(keep='first') for method in _methods
    ]
    # reduce to a df with the shared keys + all relevant scores
    lr_res = reduce(
        lambda left, right: 
        pd.merge(left, right, how='outer', on=_key_cols, suffixes=('', '_duplicated')), shared_keys
    )
    if 'Magnitude' in consensus_opts:
        lr_res[consensus.magnitude] = _rank_aggregate(lr_res.copy(),
                                                      consensus.magnitude_specs,
                                                      _key_cols,
                                                      aggregate_method=aggregate_method)
        order_col = consensus.magnitude
    if 'Specificity' in consensus_opts:
        lr_res[consensus.specificity] = _rank_aggregate(lr_res.copy(),
                                                        consensus.specificity_specs,
                                                        _key_cols,
                                                        aggregate_method=aggregate_method)
        order_col = consensus.specificity
    if 'Steady' in consensus_opts:
        lr_res[consensus.steady] = _rank_aggregate(lr_res.copy(),
                                                   consensus.steady_specs,
                                                   _key_cols,
                                                   aggregate_method=aggregate_method)
        order_col = consensus.steady
        
    lr_res = lr_res.sort_values(order_col)
    
    return lr_res

In [ ]:
aggregate(lr_res, consensus.methods, _key_cols)

In [ ]:
shared_keys = [
    lr_res[method.method_name].drop_duplicates(keep='first') for method in _methods
]
# reduce to a df with the shared keys + all relevant scores
lr_res = reduce(
    lambda left, right: 
    pd.merge(left, right, how='outer', on=_key_cols, suffixes=('', '_duplicated')), shared_keys
)

In [ ]:
lr_res[consensus.steady] = _rank_aggregate(lr_res.copy(),
                                           consensus.steady_specs,
                                           _key_cols,
                                           aggregate_method='rra')

In [ ]:
lr_res.sort_values('ranked_steady')

In [ ]:
lr_res.sort_values('ranked_steady')
# Drop duplicated

In [ ]:
rmat = _rank_aggregate(lr_res.copy(),
                       consensus.specificity_specs,
                       _key_cols,
                       aggregate_method='xx')

In [ ]:
rmat.shape

In [ ]:
consensus.specificity_specs

In [ ]:
list({consensus.magnitude_specs[s][0] for s in consensus.magnitude_specs })

In [ ]:
aggregate(lr_res, _methods, _key_cols, consensus_opts=['Steady'])

In [ ]:
# join the scores to the whole universe between the methods
shared_keys = [
    lr_res[method.method_name].drop_duplicates(keep='first') for method in _methods]
# reduce to a df with the shared keys + all relevant scores
lr_res = reduce(
    lambda left, right: pd.merge(left, right, how='outer', on=_key_cols, suffixes=('', '_duplicated')), shared_keys
)

In [ ]:
consensus.steady_specs

In [ ]:
# Set up the scores to be used (method_name:(score_name, desc))
spec_specs = {method.method_name: (
method.specificity, method.specificity_desc) for method in _methods
         if method.specificity is not None}
mag_specs = {method.method_name: method.magnitude for method in _methods
        if method.magnitude is not None}
steady_specs = spec_specs.copy()
steady_specs[sca.method_name]=(sca.magnitude, sca.magnitude_desc)

In [ ]:
_rank_aggregate(lr_res.copy(), steady_specs, _key_cols, aggregate_method='rra')

In [ ]:
specs = steady_specs

In [ ]:
# Convert specs columns to ranks
for spec in specs:
    score_name = specs[spec][0]
    desc = specs[spec][1]

    if desc:
        lr_res[score_name] = rankdata(lr_res[score_name] * -1, method='average')
    else: 
        lr_res[score_name] = rankdata(lr_res[score_name], method='average')


In [ ]:
lr_res

In [ ]:

# keep only the ranks (joins order the keys)
rmat = lr_res.drop(_key_cols, axis=1).values

In [ ]:
np.mean(rmat, axis=1)

Robust rank aggregate

In [ ]:
keys = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [ ]:
# Duplicates arrise due to the explode of complexes, in which subunits have the same min expression
# these are kept when returning the result from each method but are discarded 
# when working with the consensus because duplicated interactions are assigned the same score

In [ ]:
from functools import reduce

In [ ]:
from scipy.stats import rankdata

In [ ]:
from scipy.stats import beta

In [ ]:
def _rho_scores(rmat, dist_a, dist_b):
    # Sort values by sources (rows)
    rmat = np.sort(rmat, axis=1)
    # Calc beta cdf across rows
    p = beta.cdf(rmat, dist_a, dist_b)
    # get min pval per row 
    p = np.min(p, axis=1)
    # correct p-vals 
    rho = _corr_beta_pvals(p, k = rmat.shape[1])
    
    return rho

In [ ]:
def _corr_beta_pvals(p, k):
    p = np.clip(p * k, a_min=0, a_max=1)
    return p

In [ ]:
# 1. Get all keys

In [ ]:
# get all shared keys
shared_keys = [lr_res[method.method_name][keys].drop_duplicates(keep='first') for method in _methods]
# outer join all from different methods to keep all possibilities
shared_keys = reduce(lambda left, right: pd.merge(left, right, how='outer', on=keys), shared_keys)

In [ ]:
# 2. Set-up the scores to be used (method_name:(score_name, desc))
specs = {method.method_name:(method.specificity, method.specificity_desc) for method in _methods if method.specificity is not None}
mags = {method.method_name:method.magnitude for method in _methods if method.magnitude is not None}
steady = specs + {sca.method_name:sca.magnitude}

In [ ]:
# steady
specs[sca.method_name]=(sca.magnitude, sca.magnitude_desc)

In [ ]:
specs

^ Up to here in liana-pipe

In [ ]:
# 3. To them join the scores

In [ ]:
res = [ lr_res[spec][keys + [specs[spec][0]]].drop_duplicates(keep='first') for spec in specs ]

In [ ]:
# reduce to a df with the shared keys + all relevant scores
res = reduce(lambda left, right: pd.merge(left, right, how='left', on=keys), [shared_keys] + res)

In [ ]:
res

In [ ]:
# 4. Calculate Ranks

In [ ]:
# Convert specs columns to ranks
for spec in specs:
    score_name = specs[spec][0]
    desc = specs[spec][1]
    
    if desc:
        res[score_name] = rankdata(res[score_name] * -1, method='average')
    else: 
        res[score_name] = rankdata(res[score_name], method='average')


Get Ranked Matrix

In [ ]:
rmat = res.drop(keys, axis=1).values

Mean rank

In [ ]:
def _mean_rank(rmat):
    return np.mean(rmat, axis=1)

Re-implement robust rank

In [ ]:
def _robust_rank_aggregate(rmat):
    # 0-1 values depending on releative rank of
    # each interaction divided by the max of each method
    # due to max diffs due to ties
    rmat = rmat / np.max(rmat, axis=0)
    # generate dist_a/b with same row size as rmat
    dist_a = np.repeat([np.arange(rmat.shape[1])], rmat.shape[0], axis=0) + 1
    dist_b = rmat.shape[1] - dist_a + 1
    
    return _rho_scores(rmat, dist_a, dist_b)

In [ ]:
_robust_rank_aggregate(rmat)

In [ ]:

rmat = res.drop(keys, axis=1).values / np.max(rmat, axis=0) ## DIVIDE BY THE MAX OF EACH NOT BY THE LENGTH (to deal /w min_rank stuff)

In [ ]:
rmat

In [ ]:
dist_a = np.repeat([np.arange(rmat.shape[1])], rmat.shape[0], axis=0) + 1

In [ ]:
dist_b = rmat.shape[1] - dist_a + 1

In [ ]:
np.sort(rmat[2474])

In [ ]:
# for 1 row (interaction)
corr_beta_pvals(np.min(beta.cdf(np.sort(rmat[2474]), dist_a[1], dist_b[1]), axis=0), k = rmat.shape[1])

In [ ]:
_rho_scores(rmat, dist_a, dist_b)

In [ ]:
rmat[2474]

In [ ]:
res.iloc[2474]

In [ ]:
shared_keys['aggregate_rank'] = rho

In [ ]:
shared_keys[(shared_keys['ligand_complex']=='COL18A1') & (shared_keys['receptor_complex']=='ITGA5') ].sort_values('mean_rank')

In [ ]:
shared_keys[shared_keys['aggregate_rank']==0]

In [ ]:
shared_keys.sort_values('aggregate_rank')

In [ ]:
# test rho_scores
a = np.array([0.1, 0.3, 0.5, 0.1, 0.3])
x = beta.cdf(np.sort(a), dist_a[1], dist_b[1])
corr_beta_pvals(np.min(x), 5)
# lianaR = .rho_scores(c(0.1, 0.3, 0.5, 0.1, 0.3))

In [ ]:
import numpy as np
import pandas as pd

from numpy.random import default_rng

from scipy.stats import beta


def beta_scores(rmat):
    rmat = np.sort(rmat, axis=0)
    n = rmat.shape[0]
    dist_a = np.repeat([np.repeat([np.arange(n)], rmat.shape[1], axis=0)], rmat.shape[2], axis=0).T + 1
    dist_b = n - dist_a + 1
    p = beta.cdf(np.sort(rmat, axis=0), dist_a, dist_b)
    return p


def corr_beta_pvals(p, k):
    p = np.clip(p * k, a_min=0, a_max=1)
    return p

In [ ]:
for method in _methods:
    if method.specificity is not None:
        # relevant_cols = [x for x in keys + [method.specificity, method.magnitude] if x is not None]
        lr = lr_res[method.method_name][keys + [method.specificity]]
        # discard any duplicates
        # this occurs when e.g. multiple subunits with expr. of 0 exist
        lr = lr.drop_duplicates(keep='first')
        lr_ranks[method.method_name] = lr
        # order = array.argsort()
        # ranks = order.argsort()
        # lr_ranks[method.method_name] = ranks


In [ ]:
joined = reduce(lambda left, right: pd.merge(left, right, how='outer', on=keys), lr_ranks.values())
specs = joined[['pvals', 'scaled_weight', 'lr_logfc', 'spec_weight']].values.T

In [ ]:
# Rank thing
rmat1 = ((specs.shape[1] - np.argsort(np.argsort(specs))) / (specs.shape[1])).T

In [ ]:
rmat

In [ ]:
# Desc:True/False; Method_name:Score, specificity, magnitude, steady (specificity + sca)

In [ ]:
lr_ranks={}

In [ ]:
for method in _methods:
    if method.magnitude is not None:
        # relevant_cols = [x for x in keys + [method.specificity, method.magnitude] if x is not None]
        lr = lr_res[method.method_name][keys + [method.magnitude]]
        # discard any duplicates
        # this occurs when e.g. multiple subunits with expr. of 0 exist
        lr = lr.drop_duplicates(keep='first')
        lr_ranks[method.method_name] = lr
        # order = array.argsort()
        # ranks = order.argsort()
        # lr_ranks[method.method_name] = ranks

In [ ]:
joined = reduce(lambda left, right: pd.merge(left, right, how='outer', on=keys), lr_ranks.values())

In [ ]:
joined # issue with non-unique: should iterateve over unique scores, not methods

In [ ]:
mags = joined[['lr_means_x', 'expr_prod_x', 'expr_prod_y','lr_means_y']].values.T
# Rank thing
rmat2 = ((mags.shape[1] - np.argsort(np.argsort(mags))) / (mags.shape[1])).T

In [ ]:
rmat = np.sort(rmat, axis=0)
n = rmat.shape[0]

In [ ]:
np.repeat(np.empty(1), 100)

In [ ]:
?np.repeat

In [ ]:
any(lr_res['CellPhoneDB'][keys].duplicated())

In [ ]:
any(joined.duplicated())

In [ ]:
lr_ranks['CellPhoneDB'][lr_ranks['CellPhoneDB'][keys].duplicated()]

In [ ]:
any(joined.duplicated())

In [ ]:
joined.drop_duplicates(keep='first', inplace=False)

In [ ]:
joined

In [ ]:
lr_res['CellPhoneDB']['x'] = 1

In [ ]:
all(joined.merge(lr_res['CellPhoneDB'], how='left')['x']==1)

In [ ]:
any(lr_res['CellPhoneDB'].duplicated())

In [ ]:
joined[joined[keys].duplicated()]

In [ ]:
any(np.isnan(joined['scaled_weight']))

In [ ]:
lr_ranks.values

In [ ]:
lr_ranks['CellPhoneDB']

In [ ]:
lr_ranks = {}

In [ ]:
np.zeros([5, 1000, 1000])

In [ ]:
for method in _methods:
    if method.specificity is not None:
        lr_ranks['specs'][method.method_name] = lr_res[method.method_name][method.specificity].values
    if method.magnitude is not None:
        lr_ranks['mags'][method.method_name] = lr_res[method.method_name][method.specificity].values

In [ ]:
lr_ranks

In [ ]:
l

## Each Method

CellPhoneDB

In [ ]:
from liana import cellphonedb

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

In [ ]:
lr_res = adata.uns['liana_res']

In [ ]:
lr_res[lr_res['receptor_complex']=='CD8A_CD8B']

NATMI

In [ ]:
from liana import natmi

In [ ]:
adata = natmi(adata, groupby='label')

In [ ]:
adata.uns['liana_res']

SCA

In [ ]:
from liana import singlecellsignalr as sca

In [ ]:
adata = sca(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_means', ascending=True)

Connectome

In [ ]:
from liana import connectome

In [ ]:
adata = connectome(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('scaled_weight', ascending=False)

logFC

In [ ]:
from liana import logfc

In [ ]:
adata = logfc(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_logfc', ascending=False)

In [ ]:
np.exp(1)

try w/ real data

In [ ]:
adata = sc.datasets.pbmc3k_processed()
labels = adata.obs.louvain
cells = adata.obs_names

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata = adata[[x in cells for x in adata.obs_names]]

In [ ]:
adata.obs['label'] = labels

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

## LIANA PIPE LINE BY LINE

In [ ]:
# I get this from the score object
_complex_cols = ['ligand_means', 'receptor_means']
# change to full list and move to _var
_add_cols = ['ligand', 'receptor',
             'ligand_means_sums', 'receptor_means_sums',
             'ligand_zscores', 'receptor_zscores',
             'ligand_logfoldchanges', 'receptor_logfoldchanges',
             'ligand_logfc', 'receptor_logfc'
             'mat_mean',
             ]

_key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [ ]:
from liana.utils.pre import check_adata, check_if_covered, format_vars, filter_resource
from liana.resource import select_resource
from liana.resource.select_resource import explode_complexes
from liana.steady.liana_pipe import _get_lr
from liana.utils.reassemble_complexes import reassemble_complexes
from scipy.sparse import csr_matrix
from liana.steady._permutations import get_means_perms

I need to double check if every step makes sense, as I build unit tests!!!

For example, are there any duplicated rows (source, target, ligand_complex, receptor_complex)

Double check again if correct subunit is kept, etc

In [ ]:
adata = check_adata(adata, True)

In [ ]:
# Define idents col name
adata.obs.label = adata.obs[groupby]

Load resource

In [ ]:
resource = select_resource(resource_name='consensus')

In [ ]:
# Decomplexify
resource = explode_complexes(resource)

In [ ]:
# Filter Resource
resource = filter_resource(resource, adata.var_names)

Create entities

In [ ]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [ ]:
check_if_covered(entities, adata.var_keys)

Get global mean for SCA before filtering

In [ ]:
if 'mat_mean' in _add_cols: # SHOULD BE METHOD NAME!
    adata.uns['mat_mean'] = np.mean(adata.X)

In [ ]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

Get Stats

In [ ]:
lr_res = _get_lr(adata, resource, key_cols + _complex_cols + _add_cols, de_method)

In [ ]:
lr_res

Recomplexify

In [ ]:
# If I want to implement anything else but the min as complex policy, I would need to change this:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [ ]:
lr_res = reassemble_complexes(lr_res, _key_cols, _complex_cols)

In [ ]:
lr_res

### SCORES

In [ ]:
from liana.steady.Method import Method, MethodMeta
from liana.steady.scores.cellphonedb import _cpdb_score

CELLPHONEDB

In [ ]:
_cellphonedb = MethodMeta(method_name = "cellphonedb",
                          complex_cols = ['ligand_means', 'receptor_means'], 
                          add_cols = ['ligand', 'receptor'],
                          fun = _cpdb_score,
                          magnitude = 'lr_means',
                          specificity = 'pvals',
                          permute = True,
                          reference = 'Efremova et al., 2020')

In [ ]:
cellphonedb = Method(_SCORE=_cellphonedb)

In [ ]:
adata = cellphonedb(adata, groupby, seed=666)

In [ ]:
adata.uns['liana_res']

logFC re-implement

Magnitude as None

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

Both Specificity and Magnitude

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)